# Importing Libraries

In [ ]:
!pip3 install googletrans==3.1.0a0
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 5.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=6b9701b9aafe30a0e161366b51fb3e4e1f37d8fe5dbcba299313498db7427013
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
 

In [ ]:
import json
import pandas as pd
import os
import gzip
import pickle
from pathlib import Path
from os import listdir
from os.path import isfile, join
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm
import googletrans
from pprint import pprint


import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Helper

In [ ]:
nlp = spacy.load('en_core_web_sm')


def path_of(location):
    me_dir, me_file= os.path.split(os.path.abspath(__file__))
    return os.path.join(me_dir, location)


def get_all_files(location, get_full_path= False):
    location= path_of(location)
    onlyfiles = [f for f in listdir(location) if isfile(join(location, f))]
    if get_full_path:
        onlyfiles= [join(location, f) for f in onlyfiles]
    return onlyfiles


def load_pkl(filename):
    filename= filename
    data= None
    with open(filename, "rb") as handle:
        data= pickle.load(handle)
        handle.close()
    return data

def store_pkl(object, filename):
    filename= filename
    with open(filename, "wb") as handle:
        pickle.dump(object, handle, protocol=pickle.HIGHEST_PROTOCOL)
        handle.close()

def is_valid_file(filename):
    filename= path_of(filename)
    file= Path(filename)
    if file.is_file():
        return True
    return False


def to_same_shape(arr_of_items, required_shape):
    if len(arr_of_items) == 0:
        print("Error: tried to make shape: ", required_shape, ", but item is empty...")
        exit()
    if len(arr_of_items)>required_shape:
        return arr_of_items[:required_shape]
    res= []
    ind=0
    while len(res)<required_shape:
        res.append(arr_of_items[ind])
        ind= (ind+1)%len(arr_of_items)
    return res

# Meta Information encoder

In [ ]:
from tqdm import tqdm
import numpy as np

#import helpers

vector_file= "/content/drive/MyDrive/sony_IITPatna/word_vector_300d.vec"
hindi = "/content/drive/MyDrive/Frames/cc.hi.300.vec"

def get_unique_words(ctnr:set, phases:list):
    for words in phases:
        for word in words:
            if len(word) > 0:
                ctnr.add(word)

def get_vector(word_vec:dict, words:set):
    with open(vector_file) as file:
        for line in tqdm(file):
            word_and_vals= line.split(" ")
            w= word_and_vals[0]
            if w in words:
                vec= word_and_vals[1:]
                vec= list(map(float, vec))
                vec= np.array(vec, dtype=float)
                word_vec.update({
                    w: vec
                })
    file.close()

def split_by_words(arr:list):
    res= []
    for ele in arr:
        words= ele.split(" ")
        r= list()
        for w in words:
            if len(w)>0:
                r.append(w)
        res.append(r)
    return res

def avg_vec(words_arr:list, word_vec:dict):
    res= np.zeros(300)
    found_words= 0
    for w in words_arr:
        if not word_vec.get(w) is None:
            res= res+word_vec.get(w)
            found_words+=1
    if found_words>0:
        res= res/found_words
    return res


def transE_single(head, relation, tail, word_vec):
    head_v= avg_vec(head, word_vec)
    relation_v= avg_vec(relation, word_vec)
    tail_v= avg_vec(tail, word_vec)
    enc= head_v+relation_v-tail_v
    return enc


def transE(head:list, relation:list, tail:list, word_vec:dict):
    if len(head) != len(relation) or len(head)!= len(tail):
        raise "\n\tError!! head, tail, relation doesnot have same length\n"
    res= []
    for index in range(len(head)):
        vec= transE_single(head[index],relation[index], tail[index], word_vec)
        res.append(vec)
    res= np.array(res)
    return res

def encode(head:list, relation:list, tail:list):
    unique_words= set()

    head= split_by_words(head)
    relation= split_by_words(relation)
    tail= split_by_words(tail)

    get_unique_words(unique_words, head)
    get_unique_words(unique_words, relation)
    get_unique_words(unique_words, tail)

    word_vec= dict()
    get_vector(word_vec, unique_words)

    res= transE(head, relation, tail, word_vec)
    return res

In [ ]:
def get_entities(sent):
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""
    prv_tok_text = ""

    prefix = ""
    modifier = ""


    for tok in nlp(sent):
        if tok.dep_ != "punct":
            if tok.dep_ == "compound":
                prefix = tok.text
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text

            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text

            if tok.dep_.find("subj") == True:
                ent1 = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""

            if tok.dep_.find("obj") == True:
                ent2 = modifier +" "+ prefix +" "+ tok.text

            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text

    return [ent1.strip(), ent2.strip()]


def get_relation(sent):
    doc = nlp(sent)

    matcher = Matcher(nlp.vocab)

    pattern =           [{'DEP':'ROOT'},
                        {'DEP':'prep','OP':"?"},
                        {'DEP':'agent','OP':"?"},
                        {'POS':'ADJ','OP':"?"}]

    matcher.add("matching_1", None, pattern)
    matches = matcher(doc)
    k = len(matches) - 1
    span = doc[matches[k][1]:matches[k][2]]
    return(span.text)


def get_elements(sent_arr):
    head= []
    tail= []
    relation= []

    for sen in tqdm(sent_arr):
        try:
            entity= get_entities(sen)
            rel= get_relation(sen)
        except:
            entity= ["None", "None"]
            rel= "None"
        head.append(entity[0])
        tail.append(entity[1])
        relation.append(rel)
    return head, relation, tail


def draw_graph(head, relation, tail):
    kg_df = pd.DataFrame({'source':head, 'target':tail, 'edge':relation})
    G=nx.from_pandas_edgelist(kg_df, "source", "target", edge_attr=True, create_using=nx.MultiDiGraph())
    plt.figure(figsize=(12,12))
    pos = nx.spring_layout(G)
    nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
    plt.show()


def inner_preprocess_sen(sen:str):
    sen= sen.rstrip("\n").lstrip(" ").rstrip(" ").lstrip("\"").rstrip("\"")
    return sen

def preprocess_sen(sen):
    if type(sen) == str:
        return inner_preprocess_sen(sen)
    elif type(sen) == list:
        res= list()
        for s in sen:
            res.append(inner_preprocess_sen(s))
        return res
    else:
        raise "\n\tPreprocess sen takes string or list of string...\n"

def build_and_encode(sentence_arr:list):
    sen_arr= []
    for sen in sentence_arr:
        sen= preprocess_sen(sen)
        sen_arr.append(sen)
    h,r,t= get_elements(sen_arr)
    #import kg_encoder as kg_encoder
    encoded= encode(h,r,t)
    return encoded

# Extract Meta

In [ ]:
with open('/content/drive/MyDrive/fickscore/movies.json') as f:
    data = [json.loads(line) for line in f]

In [ ]:
director = []
rating = []
cast = []
summary = []
movie_id = []
name = []

In [ ]:
genere = []
c = 0
for i in data:
  c = c + 1
  genere.append(i['genre']) # Making list of genres of all movies

In [ ]:
final_genere = []        # Doing one hot encoding of genres of all movies
d = 0
for j in genere:

  l = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for i in j:
    if i == 'Biography':
         l[0] = 1
    if i == 'Musical':
         l[1] = 1
    if i == 'Mystery':
         l[2] = 1
    if i == 'Adventure':
         l[3] = 1
    if i == 'Action':
         l[4] = 1
    if i == 'War':
         l[5] = 1
    if i == 'Music':
         l[6] = 1
    if i == 'Fantasy':
         l[7] = 1
    if i == 'Romance':
         l[8] = 1
    if i == 'Thriller':
         l[9] = 1
    if i == 'Family':
         l[10] = 1
    if i == 'History':
         l[11] = 1
    if i == 'Crime':
         l[12] = 1
    if i == 'Drama':
         l[13] = 1
  d = d + 1
  final_genere.append(l)

In [ ]:
for i in data:
  #if i['movie_id'] in dict1:
      director.append(i['director'])
      rating.append(i['rating'])
      cast.append(i['cast'])
      summary.append(i['description'])
      movie_id.append(i['movie_id'])
      name.append(i['name'])

In [ ]:
"""
Creating a dictionary to store meta information of movies in the following format:
{
  "name": movie name,
  "genre": movie genres,
  "director": movie director,
  "rating": movie rating,
  "casts": movie casts,
  "summary": movie summary,
}

"""
op_dict = dict()
count = 0
for i in range(2850):
  #print(director[i])
  if len(director[i])!= 0 :
     dir = director[i][0]
  else:
    dir = "None"
  if len(dir)==0:
            print("skiping_for_director")
            continue
  rat = rating[i]
  if len(rating)==0:
            print("skiping_for_ratings")
            continue
  genres = final_genere[i]
  rat = rating[i]
  nam = name[i]
  casts = cast[i]
  if len(casts) == 0:
            print("skiping_for_casts")
            continue
  # casts = cast[i]
  summ = summary[i]
  id = movie_id[i]
  if len(summary[i])!=0 and len(genres)!=0  and len(cast[i])!=0:
    count = count + 1
    op_dict.update({
            id: {
                "name": nam,
                "genre": genres,
                "director": dir,
                "rating": rat,
                "casts": casts,
                "summary": summ,
            }
        })

skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts
skiping_for_casts


# Process Meta

In [ ]:
meta_raw= op_dict

translator = googletrans.Translator()

meta_processed_op_oath= "latest_sentence_fickscore_meta_processed_v2.obj" # Path where you want to save your textual embedding

w2v_file_path = "/content/drive/MyDrive/sony_IITPatna/word_vector_300d.vec"

hindi = "/content/drive/MyDrive/Frames/cc.hi.300.vec"

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

lines_to_take= 3


def filter_and_preprocess_lines(lines_arr):
    res= list()
    for l in lines_arr:
        l= l.lstrip(" ").rstrip(" ")
        if len(l)==0: continue
        if len(l.split(" "))<4: continue
        res.append(l)
    return res

def concat_encoded(encoded_arr):
    res= list()
    for item in encoded_arr:
        res.extend(item)
    return res

def avg_encode(encoded_arr):
    res= [0 for x in range(len(encoded_arr[0]))]
    for enc in encoded_arr:
        for ind, val in enumerate(enc):
            res[ind]+=val
    for index in range(len(res)):
        res[index]= res[index]/len(encoded_arr)
    return res

def encode_description(meta_raw):

    """
    Function to encode movie description
    Input: Meta information
    Output: Encoded vector of length 384
    """

    all_lines= []
    all_movie_id= list(meta_raw.keys())
    res = dict()
    # print(all_movie_id)

    lines_to_take_while_encoding= 8

    for movie_id in all_movie_id:
        movie_meta= meta_raw.get(movie_id)
        description= movie_meta.get("summary")
        lines_raw= description.split(".")
        lines_filtered= filter_and_preprocess_lines(lines_raw)
        encoded_lines = model.encode(lines_filtered)
        final_encoded = np.average(encoded_lines,axis=0)
        res.update({
            movie_id: final_encoded,
        })

    return res


def encode_director(meta_raw):

    """
    Function to encode director name
    Input: Meta information
    Output: Encoded vector of length 300
    """

    all_movie_id= list(meta_raw.keys())

    director_split_vectors= dict()

    for movie_id in all_movie_id:
        movie_meta= meta_raw.get(movie_id)
        director_name= movie_meta.get("director")
        results = translator.translate(director_name,dest='hi')
        director_split= results.text.split(" ")
        for d in director_split:
            d= d.lower()
            director_split_vectors.update({
                d: False
            })

    with open(hindi) as file:
        for line in file:
            line= line.rstrip("\n")
            items= line.split(" ")
            word= items[0].lower()
            if director_split_vectors.get(word) is not None:
                vector= [float(x) for x in items[1:]]
                director_split_vectors.update({
                    word: vector,
                })
        file.close()

    director_encoding = dict()

    for movie_id in all_movie_id:
        movie_meta= meta_raw.get(movie_id)
        director_name= movie_meta.get("director")
        results = translator.translate(director_name,dest='hi')
        director_split= results.text.split(" ")
        enc= list()
        for d in director_split:
            d= d.lower()
            if director_split_vectors.get(d):
                enc.append(director_split_vectors.get(d))
            else:
                enc.append(list(np.random.uniform(-0.2,0.2,300)))

        enc= to_same_shape(enc, 2)
        enc= avg_encode(enc)

        director_encoding.update({
            movie_id: enc,
        })

    return director_encoding

def encode_movie_name(meta_raw):

    """
    Function to encode movie name
    Input: Meta information
    Output: Encoded vector of length 300
    """

    all_movie_id= list(meta_raw.keys())

    movie_split_vectors= dict()

    for movie_id in all_movie_id:
        movie_meta= meta_raw.get(movie_id)
        movie_name= movie_meta.get("name")
        results = translator.translate(movie_name,dest='hi')
        movie_split= results.text.split(" ")
        for d in movie_split:
            d= d.lower()
            movie_split_vectors.update({
                d: False
            })

    with open(hindi) as file:
        for line in file:
            line= line.rstrip("\n")
            items= line.split(" ")
            word= items[0].lower()
            if movie_split_vectors.get(word) is not None:
                vector= [float(x) for x in items[1:]]
                movie_split_vectors.update({
                    word: vector,
                })
        file.close()

    movie_encoding = dict()

    for movie_id in all_movie_id:
        movie_meta= meta_raw.get(movie_id)
        movie_name= movie_meta.get("name")
        results = translator.translate(movie_name,dest='hi')
        movie_split= results.text.split(" ")
        enc= list()
        for d in movie_split:
            d= d.lower()
            if movie_split_vectors.get(d):
                enc.append(movie_split_vectors.get(d))
            else:
                enc.append(list(np.random.uniform(-0.2,0.2,300)))

        enc= to_same_shape(enc, 2)
        enc= avg_encode(enc)
        movie_encoding.update({
            movie_id: enc,
        })
    return movie_encoding

def encode_genre(meta_raw):

    """
    Function to encode movie genre
    Input: Meta information
    Output: One hot encoded vector of length 15
    """


    all_movie_id= list(meta_raw.keys())
    genre_encoded= dict()
    for movie_id in all_movie_id:
        movie_meta= meta_raw.get(movie_id)
        genre= movie_meta.get("genre")
        genre_encoded.update({
            movie_id: genre,
        })
    return genre_encoded


def encode_rating(meta_raw):

    """
    Function to normalize movie rating
    Input: Meta information
    Output: 1 single float value
    """
    all_movie_id= list(meta_raw.keys())
    rating_encoded= dict()
    for movie_id in all_movie_id:
        movie_meta= meta_raw.get(movie_id)
        rating= movie_meta.get("rating")
        rating= rating/10
        rating_encoded.update({
            movie_id: [rating],
        })
    return rating_encoded


def encode_everything():

    """
    Function to encode description, movie name, director, genre, and rating
    Input: Meta information
    Output: Encoded vector of dimension 999 (14 + 1 + 300 + 300 + 384)
    """

    enc_desc= encode_description(meta_raw)
    print("[*] Description encoding completed...\n")
    enc_name= encode_movie_name(meta_raw)
    print("[*] Name encoding completed...\n")
    enc_direc= encode_director(meta_raw)
    print("[*] Director encoding completed...\n")
    enc_genre= encode_genre(meta_raw)
    print("[*] Genre encoding completed...\n")
    enc_rate= encode_rating(meta_raw)
    print("[*] Rating encoding completed...\n")

    metadata_encoded= dict()

    all_movie_id= list(meta_raw.keys())
    for movie_id in all_movie_id:
        de= enc_desc.get(movie_id)
        ge= enc_genre.get(movie_id)
        di= enc_direc.get(movie_id)
        ra= enc_rate.get(movie_id)
        na = enc_name.get(movie_id)

        enc_concated= list()
        enc_concated.extend(de)
        enc_concated.extend(ge)
        enc_concated.extend(di)
        enc_concated.extend(ra)
        enc_concated.extend(na)

        metadata_encoded.update({
            movie_id: enc_concated,
        })

    store_pkl(metadata_encoded, meta_processed_op_oath)
    return metadata_encoded,enc_desc

In [ ]:
a = encode_everything()

[*] Description encoding completed...

[*] Name encoding completed...

[*] Director encoding completed...

[*] Genre encoding completed...

[*] Rating encoding completed...



In [ ]:
import pickle
file_pi = open('/content/drive/MyDrive/sony_IITPatna/latest_text_emvedding.obj', 'wb')
pickle.dump(a, file_pi)

In [ ]:
bnm = load_pkl("/content/drive/MyDrive/sony_IITPatna/latest_text_emvedding.obj")

In [ ]:
len(bnm['tt1520485'])